In [13]:
import utils
import feature_extractor
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

# Constant Definition

In [14]:
NUM_CLASSES=19
BATCH_SIZE=64
EPOCHS=10
HIDDEN_SIZE=64
VECTOR_SIZE = 50  # Size of word vectors
WINDOW_SIZE = 5  # Context window size
THREADS = 4  # Number of threads to use for training
CUTOFF_FREQ = 1  # Minimum frequency for a word to be included in vocabulary

# reading dataset

In [15]:
complete_data=utils.read_data("../data/fixed_PIZZA_train.json")
data = complete_data[:100000]
corpus,top_tokenized,dec_tokenized= utils.get_train_dataset(data)

In [16]:
entites_output_as_number_labels,intents_output_as_number_labels=utils.label_complete_input(corpus,top_tokenized,dec_tokenized)
input_as_tokenized_string=feature_extractor.list_of_lists(corpus)

# Embedding model

In [17]:
emb_model = feature_extractor.train_gensim_w2v_model(corpus)

# NER Model

In [18]:
class LargeDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data 
        self.labels = labels 
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

def collate_fn(batch):
    sequences, labels = zip(*batch)
    #I believe we can transform words into embeddings here
    embeddings=[]
    # print(sequences)
    for seq in sequences:
        x=[]
        for token in seq:
            x.append(emb_model.wv[token])
        embeddings.append(x)
    sequences=embeddings
    labels = [torch.tensor(label, dtype=torch.long) for label in labels]
    padded_labels = pad_sequence(labels, batch_first=True, padding_value=-1)
    sequences = [torch.tensor(seq) for seq in sequences]
    padded_sequences = pad_sequence(sequences, batch_first=True)
    lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)
    return padded_sequences, padded_labels, lengths

class LargeWordRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LargeWordRNN, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
    
    def forward(self, x, lengths):
        packed_x = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, _ = self.rnn(packed_x)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)
        out = self.fc(out)
        return out

# Checking if CUDA is working

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available()) 
print(torch.version.cuda)
print(f'Using device: {device}')

True
12.4
Using device: cuda


# Training Model

In [ ]:


labels = entites_output_as_number_labels

dataset = LargeDataset(input_as_tokenized_string, labels)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True, num_workers=0)

model = LargeWordRNN(input_size=VECTOR_SIZE, hidden_size=HIDDEN_SIZE, num_classes=NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(EPOCHS): 
    for padded_sequences, padded_labels, lengths in dataloader:
        padded_sequences=padded_sequences.to(device)
        padded_labels=padded_labels.to(device)
        lengths=lengths.to(device)
        optimizer.zero_grad()
        outputs = model(padded_sequences, lengths)
        loss = criterion(outputs.view(-1, NUM_CLASSES), padded_labels.view(-1))
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")


C:\Users\Ghait\AppData\Local\Temp\ipykernel_19156\1783255267.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  sequences = [torch.tensor(seq) for seq in sequences]


Epoch 1, Loss: 0.0009
Epoch 2, Loss: 0.0007
Epoch 3, Loss: 0.0001
Epoch 4, Loss: 0.0001
Epoch 5, Loss: 0.0000
Epoch 6, Loss: 0.0000


In [ ]:
test_data=complete_data[100000:100010]
print(test_data)
print("---------------------------------------------")
test_corpus,_,_= utils.get_train_dataset(test_data)
print(test_corpus)
test_as_tokenized_string=feature_extractor.list_of_lists(test_corpus)

[{'train.SRC': 'can i have one personal sized pie with banana pepper and peperoni and yellow peppers', 'train.EXR': '(ORDER (PIZZAORDER (NUMBER 1 ) (SIZE PERSONAL_SIZE ) (TOPPING BANANA_PEPPERS ) (TOPPING PEPPERONI ) (TOPPING YELLOW_PEPPERS ) ) )', 'train.TOP': '(ORDER can i have (PIZZAORDER (NUMBER one ) (SIZE personal sized ) pie with (TOPPING banana pepper ) and (TOPPING peperoni ) and (TOPPING yellow peppers ) ) )', 'train.TOP-DECOUPLED': '(ORDER (PIZZAORDER (NUMBER one ) (SIZE personal sized ) (TOPPING banana pepper ) (TOPPING peperoni ) (TOPPING yellow peppers ) ) )'}, {'train.SRC': 'a pie with american cheese and extra meatball', 'train.EXR': '(ORDER (PIZZAORDER (NUMBER 1 ) (TOPPING AMERICAN_CHEESE ) (COMPLEX_TOPPING (QUANTITY EXTRA ) (TOPPING MEATBALLS ) ) ) )', 'train.TOP': '(ORDER (PIZZAORDER (NUMBER a ) pie with (TOPPING american cheese ) and (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING meatball ) ) ) )', 'train.TOP-DECOUPLED': '(ORDER (PIZZAORDER (NUMBER a ) (TOPPING america

In [ ]:
print(test_as_tokenized_string)

[['can', 'i', 'have', 'one', 'personal', 'sized', 'pie', 'with', 'banana', 'pepper', 'and', 'peperoni', 'and', 'yellow', 'peppers'], ['a', 'pie', 'with', 'american', 'cheese', 'and', 'extra', 'meatball'], ['party', '-', 'sized', 'pizza', 'with', 'green', 'olive', 'and', 'peperonni', 'and', 'a', 'lot', 'of', 'sauce'], ['i', 'want', 'one', 'lunch', '-', 'size', 'pizza', 'without', 'any', 'green', 'olives'], ['i', 'want', 'a', 'lunch', '-', 'sized', 'pizza', 'without', 'any', 'chorrizo'], ['pizza', 'with', 'no', 'chorizo'], ['i', 'want', 'a', 'big', 'new', 'yorker', 'pie', 'with', 'extra', 'american', 'cheese', 'and', 'with', 'pecorino'], ['one', 'party', 'sized', 'combination', 'pie', 'with', 'american', 'cheese', 'and', 'with', 'pecorino', 'cheese'], ['i', "'d", 'like', 'a', 'party', 'size', 'pie', 'with', 'banana', 'peppers', 'and', 'peperonni', 'and', 'a', 'little', 'bit', 'of', 'sauce'], ['a', 'party', '-', 'sized', 'big', 'meat', 'pizza', 'with', 'extra', 'american', 'cheese', 'and'

In [ ]:
class TestLargeDataset(Dataset):
    def __init__(self, data):
        self.data = data 
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

def test_collate_fn(batch):
    sequences = zip(*batch)
    #I believe we can transform words into embeddings here
    embeddings=[]
    print(sequences)
    for seq in sequences:
        x=[]
        for token in seq:
            x.append(emb_model.wv[token])
        embeddings.append(x)
    sequences=embeddings
    sequences = [torch.tensor(seq) for seq in sequences]
    padded_sequences = pad_sequence(sequences, batch_first=True)
    lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)
    return padded_sequences, lengths


print(test_as_tokenized_string)
test_as_tokenized_string
dataset = TestLargeDataset(test_as_tokenized_string)
dataloader = DataLoader(dataset, batch_size=1, collate_fn=test_collate_fn, shuffle=False, num_workers=0)
print(dataset.data)

for padded_sequences, lengths in dataloader:
    padded_sequences=padded_sequences.to(device)
    lengths=lengths.to(device)
    outputs = model(padded_sequences, lengths)
    for out in outputs[0]:
        print(torch.argmax(out))

[['can', 'i', 'have', 'one', 'personal', 'sized', 'pie', 'with', 'banana', 'pepper', 'and', 'peperoni', 'and', 'yellow', 'peppers'], ['a', 'pie', 'with', 'american', 'cheese', 'and', 'extra', 'meatball'], ['party', '-', 'sized', 'pizza', 'with', 'green', 'olive', 'and', 'peperonni', 'and', 'a', 'lot', 'of', 'sauce'], ['i', 'want', 'one', 'lunch', '-', 'size', 'pizza', 'without', 'any', 'green', 'olives'], ['i', 'want', 'a', 'lunch', '-', 'sized', 'pizza', 'without', 'any', 'chorrizo'], ['pizza', 'with', 'no', 'chorizo'], ['i', 'want', 'a', 'big', 'new', 'yorker', 'pie', 'with', 'extra', 'american', 'cheese', 'and', 'with', 'pecorino'], ['one', 'party', 'sized', 'combination', 'pie', 'with', 'american', 'cheese', 'and', 'with', 'pecorino', 'cheese'], ['i', "'d", 'like', 'a', 'party', 'size', 'pie', 'with', 'banana', 'peppers', 'and', 'peperonni', 'and', 'a', 'little', 'bit', 'of', 'sauce'], ['a', 'party', '-', 'sized', 'big', 'meat', 'pizza', 'with', 'extra', 'american', 'cheese', 'and'

NameError: name 'emb_model' is not defined

In [ ]:
    entity_to_num = {"I_NUMBER": 0, "I_SIZE": 1, "I_TOPPING": 2, "I_STYLE": 3, "I_DRINKTYPE": 4, "I_CONTAINERTYPE": 5, "I_VOLUME": 6, "I_QUANTITY": 7, "B_NUMBER": 8, "B_SIZE": 9, "B_TOPPING": 10, "B_STYLE": 11, "B_DRINKTYPE": 12, "B_CONTAINERTYPE": 13, "B_VOLUME": 14, "B_QUANTITY": 15, "I_NOT_TOPPING": 16, "B_NOT_TOPPING": 17, "NONE": 18}
